In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
%matplotlib inline

#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_csv('C:/Hari Docs/Dataset/BigMart3_Train.csv')
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [3]:
#Continuous variable data
df.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [4]:
#Categorical variable data
cat = df.dtypes[df.dtypes == 'object'].index
df[cat].describe()
#df['Item_Identifier','Item_Fat_Content','Item_Type','Outlet_Identifier','Outlet_Size','Outlet_Location_Type','Outlet_Type'].describe()

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,8523,8523,8523,8523,6113,8523,8523
unique,1559,5,16,10,3,3,4
top,FDW13,Low Fat,Fruits and Vegetables,OUT027,Medium,Tier 3,Supermarket Type1
freq,10,5089,1232,935,2793,3350,5577


In [27]:
#Determine the average weight per item:
#item_avg_weight = df.pivot_table(values='Item_Weight', index='Item_Identifier')
#item_avg_weight
df['Outlet_Size'].isnull().sum()
#Get a boolean variable specifying missing Item_Weight values
#miss_bool = df['Item_Weight'].isnull() 
#df.loc[miss_bool, 'Item_Identifier']
#Impute data and check #missing values before and after imputation to confirm
#print ('Orignal #missing: %d'% sum(miss_bool))
#df.loc[miss_bool,'Item_Weight'] = df.loc[miss_bool,'Item_Identifier'].apply(lambda x: item_avg_weight[x])
#print ('Final #missing: %d'% sum(df['Item_Weight'].isnull()))

2410

In [7]:
#df.groupby('Outlet_Type').Outlet_Size.transform(lambda x: x.fillna(x.mode()[0]))

df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [5]:
#Replace Item_Fat_Content with appropriate values - 1
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace(['LF', 'low fat'], 'Low Fat')

In [6]:
#Replace Item_Fat_Content with appropriate values - 2
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace('reg', 'Regular')

In [32]:
#Item avg weight
#mean_wt = df.groupby(['Item_Identifier'])['Item_Weight'].mean()

#Items having Item_weight as NULL
 df['Item_Identifier'].df['Item_Weight'].isnull()



0       False
1       False
2       False
3       False
4       False
5       False
6       False
7        True
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18       True
19      False
20      False
21       True
22      False
23       True
24      False
25      False
26      False
27      False
28      False
29       True
        ...  
8493    False
8494    False
8495    False
8496    False
8497    False
8498    False
8499    False
8500    False
8501    False
8502    False
8503    False
8504     True
8505    False
8506    False
8507    False
8508    False
8509    False
8510    False
8511    False
8512    False
8513    False
8514    False
8515    False
8516    False
8517    False
8518    False
8519    False
8520    False
8521    False
8522    False
Name: Item_Weight, Length: 8523, dtype: bool